In [40]:
import io
import wave
from pydub import AudioSegment

In [41]:
def extract_wave_header_and_params(wave_bytes):
    with io.BytesIO(wave_bytes) as wave_bytes_io:
        with wave.open(wave_bytes_io, 'rb') as wave_file:
            header = wave_file.getparams()
    return header

In [46]:
with open('../data/audio_82.wav', 'rb') as f:
    audio = f.read()
    header = extract_wave_header_and_params(audio)
header

_wave_params(nchannels=1, sampwidth=2, framerate=48000, nframes=2147483625, comptype='NONE', compname='not compressed')

In [47]:
new_file_path = '../data/new_audio.wav'

with open('../data/audio_41.wav', 'rb') as f:
    chunk_of_data = f.read()  # Replace this with your new audio data

# Creating a BytesIO object to write the wave header and chunk wave bytes
with io.BytesIO() as wave_bytes_io:
    # Using wave module to write header and data into the BytesIO object
    with wave.open(wave_bytes_io, 'wb') as wave_file:
        wave_file.setparams(header)
        wave_file.writeframes(chunk_of_data)
    
    # Getting the complete wave bytes (header + data)
    complete_wave_bytes = wave_bytes_io.getvalue()

audio_bytes = io.BytesIO(complete_wave_bytes) 
audio_segment = AudioSegment.from_file(audio_bytes)
audio_segment

In [ ]:
import whisper

model = whisper.load_model('base')

In [ ]:
import numpy as np

# convert to expected format
if audio_segment.frame_rate != 16000: # 16 kHz
    audio_segment = audio_segment.set_frame_rate(16000)
if audio_segment.sample_width != 2:   # int16
    audio_segment = audio_segment.set_sample_width(2)
if audio_segment.channels != 1:       # mono
    audio_segment = audio_segment.set_channels(1)        
arr = np.array(audio_segment.get_array_of_samples())
arr = arr.astype(np.float32) / 32768.0

res = model.transcribe(arr)
res['text']

In [ ]:
# audio_bytes = io.BytesIO(message)
# audio_segment = AudioSegment.from_file(audio_bytes, format='wav')
# audio_to_transcript = convert_bytes_to_whisper_format(audio_segment)

# transcript = model.transcribe(audio_to_transcript)
# print(transcript['text'])